# Introduction

In [ ]:
using DataFrames, Random

In [3]:
DataFrame()

""
""


In [4]:
DataFrame(A = 1:3, B = rand(3), C = randstring.([3, 3, 3]), fixed = 1)

,A,B,C,fixed
,Int64,Float64,String,Int64
1,1,0.0475212,Bjs,1
2,2,0.880892,Xem,1
3,3,0.0445768,JMU,1


In [5]:
x = Dict("A" => [1, 2], "B" => [true, false], "C" => ['a', 'b'], "fixed" => Ref([1, 1]))

Dict{String, Any} with 4 entries:
  "B"     => Bool[1, 0]
  "A"     => [1, 2]
  "fixed" => RefValue{Vector{Int64}}([1, 1])
  "C"     => ['a', 'b']

what is `Ref()`

In [6]:
DataFrame(x)

,A,B,C,fixed
,Int64,Bool,Char,Array…
1,1,1,a,"[1, 1]"
2,2,0,b,"[1, 1]"


In [7]:
DataFrame(:A => [1, 2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,1,a
2,2,0,b


In [8]:
DataFrame(:A => [1, 2], :B => [true, false], :C => ['a', 'b'], :fixed => "const")

,A,B,C,fixed
,Int64,Bool,Char,String
1,1,1,a,const
2,2,0,b,const


In [9]:
[rand(3) for i in 1:3]

3-element Vector{Vector{Float64}}:
 [0.10645440855895583, 0.9112787200737531, 0.5629505472768641]
 [0.9269983602055936, 0.5100113392264543, 0.6654870521897718]
 [0.3447763548400473, 0.33530876192249104, 0.30778196823099324]

In [10]:
# :auto to generate column names auto
DataFrame([rand(3) for i in 1:3], :auto)

,x1,x2,x3
,Float64,Float64,Float64
1,0.803778,0.680064,0.865794
2,0.265424,0.187248,0.557748
3,0.282174,0.711735,0.865915


In [11]:
typeof(:a => 1)

Pair{Symbol, Int64}

In [12]:
typeof(:a => [1, 2, 3])

Pair{Symbol, Vector{Int64}}

In [13]:
DataFrame([rand(3) for i in 1:3], ["x1", "x2", "x3"])

,x1,x2,x3
,Float64,Float64,Float64
1,0.145627,0.225594,0.241678
2,0.711,0.765555,0.568409
3,0.295353,0.965073,0.950023


In [14]:
DataFrame(rand(3, 4), :auto)

,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.719238,0.997471,0.050825,0.355642
2,0.682446,0.204265,0.0842816,0.396126
3,0.514078,0.116849,0.552466,0.122724


In [15]:
DataFrame(rand(3, 4), Symbol.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.480504,0.972061,0.739771,0.888284
2,0.702104,0.765945,0.352351,0.11699
3,0.0773117,0.784038,0.737838,0.801147


In [16]:
DataFrame(rand(3, 4), string.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.50099,0.538809,0.573335,0.362154
2,0.9168,0.63159,0.528883,0.730385
3,0.65547,0.568006,0.588062,0.573417


In [17]:
string.('a':'b')

2-element Vector{String}:
 "a"
 "b"

In [18]:
DataFrame(A = Int64[], B = Float64[], C = String[])

,A,B,C
,Int64,Float64,String


In [30]:
x = DataFrame(a = 1:2, b = string.('a':'b'))

,a,b
,Int64,String
1,1,a
2,2,b


In [31]:
y = copy(x)

,a,b
,Int64,String
1,1,a
2,2,b


In [37]:
x == y, x === y, isequal(x, y), x.a == y.a, x.a === y.a

(true, false, true, true, false)

In [38]:
x

,a,b
,Int64,String
1,1,a
2,2,b


In [52]:
similar(x, 3) # 3 rows

,a,b
,Int64,String
1,140331738215856,#undef
2,140333689596736,#undef
3,140331738215872,#undef


In [53]:
typeof(x)

DataFrame

# convert to a matrix

In [54]:
Matrix(x)

2×2 Matrix{Any}:
 1  "a"
 2  "b"

In [55]:
Array(x)

2×2 Matrix{Any}:
 1  "a"
 2  "b"

In [56]:
z = DataFrame([rand(3) for i in 1:3], :auto)

,x1,x2,x3
,Float64,Float64,Float64
1,0.753646,0.922535,0.174483
2,0.415101,0.962952,0.252459
3,0.963395,0.964464,0.537673


In [57]:
Matrix{Float64}(z)

3×3 Matrix{Float64}:
 0.753646  0.922535  0.174483
 0.415101  0.962952  0.252459
 0.963395  0.964464  0.537673

In [58]:
x

,a,b
,Int64,String
1,1,a
2,2,b


In [59]:
Tables.columntable(x)

(a = [1, 2], b = ["a", "b"])

In [60]:
Tables.rowtable(x)

2-element Vector{NamedTuple{(:a, :b), Tuple{Int64, String}}}:
 (a = 1, b = "a")
 (a = 2, b = "b")

In [61]:
empty!(x)

,a,b
,Int64,String


In [62]:
y

,a,b
,Int64,String
1,1,a
2,2,b


In [63]:
for row in eachrow(y)
    println(row)
end

DataFrameRow
 Row │ a      b      
     │ Int64  String 
─────┼───────────────
   1 │     1  a
DataFrameRow
 Row │ a      b      
     │ Int64  String 
─────┼───────────────
   2 │     2  b


In [64]:
for col in eachcol(y)
    println(col)
end

[1, 2]
["a", "b"]


# basic information 

In [1]:
using DataFrames

In [2]:
x = DataFrame(A = [1, 2], B = [1.0, missing], C = ["a", "b"])

,A,B,C
,Int64,Float64?,String
1,1,1.0,a
2,2,missing,b


In [4]:
size(x), size(x, 1), size(x, 2)

((2, 3), 2, 3)

In [6]:
nrow(x), ncol(x)

(2, 3)

In [10]:
describe(x)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,A,1.5,1,1.5,2,0,Int64
2,B,1.0,1.0,1.0,1.0,1,"Union{Missing, Float64}"
3,C,,a,,b,0,String


In [11]:
describe(x, cols = 1:2)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,Type
1,A,1.5,1,1.5,2,0,Int64
2,B,1.0,1.0,1.0,1.0,1,"Union{Missing, Float64}"


In [12]:
names(x)

3-element Vector{String}:
 "A"
 "B"
 "C"

In [13]:
propertynames(x)

3-element Vector{Symbol}:
 :A
 :B
 :C

In [15]:
eltype.(eachcol(x))

3-element Vector{Type}:
 Int64
 Union{Missing, Float64}
 String

In [ ]:
y = DataFrame(rand(1:10, 1000, 10), :auto)

# slicing

# groupby